In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import pandas as pd
from gensim.models import Doc2Vec
workdir_path = '/content/drive/My Drive/' + '00 PUC BI MASTER/00 - PROJ (TCC)/PREDIÇÃO ATIVIDADES/event2mind'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

MODEL LOAD - INPUT

In [3]:
#Parametros de Input
model = Doc2Vec.load("Event2Mind_Events.model")
input_description = pd.read_csv('embeddingsEvents.csv', sep = ';')

campo_input = "Event"

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


OUTPUT - CLUSTER


In [4]:
#Parametros Output
cluster_output = pd.read_csv('clusterIntencoes.csv', sep = ',')
cluster_output.rename(columns={"Xintent": "Xintent_2"}, inplace = True)
cluster_output.rename(columns={"cluster_intents": "cluster_output"}, inplace = True)

campo_output = "Xintent_2"

TRATAMENTOS BASE

In [6]:
input_description['input_index'] = list(range(len(input_description.index)))
input_description['input_index'] = input_description['input_index'] + 1
embeddings_input = model.docvecs.vectors_docs

def removeCaracteres(nomeColuna, dataSet):
  dataSet[nomeColuna]= dataSet[nomeColuna].astype(str)
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` ve", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` s", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("'", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("&", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace(",", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("  ", " ")


RODAR ABAIXO CASO QUEIRA CRIAR UMA CLASSE COM BASE NAS PALAVRAS

In [ ]:
'''
listPalavras = ['get', 'make', 'help', 'see', 'show', 'know', 'go', 'give', 'take', 'keep', 'fell', 'enjoy', 'work', 'wants', 'find', 'look', 'avoid']
#listPalavras = ['happy', 'glad', 'excited', 'relieved', 'show', 'know', 'go', 'give', 'take', 'keep', 'fell', 'enjoy', 'work', 'wants', 'find', 'look', 'avoid']
listOutput = dtPrincipal[campo_output].toList()

import math

classes = []

for i in range(0, len(listOutput):
  texto = listOutput[i]
  classeTemp = []
  for z in range(0, len(listPalavras)):
       if (texto.find(listPalavras[z]) != -1):
        flag = z
        classeTemp.append(flag)
         
    
  classeFinal = pd.Series(classeTemp).min()
  classes.append(classeFinal)

maximo = pd.Series(classes).max()    
classes = [ maximo + 1 if math.isnan(x) else x for x in classes] 

cluster_output = cluster_output.assign(classes=classes)

CRUZANDO CLUSTER DE INTEÇÕES COM EVENTOS PARA DEIXA-LOS NA MESMA DIMENSÃO

In [8]:
#BASE COM TODAS AS INTENÇÕES E EMOÇÕES
dtPrincipal = pd.read_csv('event2MindClean.csv', sep = ',')
dtPrincipal = dtPrincipal[dtPrincipal[campo_output] != "none"]
dtPrincipal = dtPrincipal[dtPrincipal[campo_output] != "'none'"]
removeCaracteres(campo_output, dtPrincipal)

#set_index = input variable
dtPrincipal = dtPrincipal.join(input_description.set_index(campo_input), on = campo_input, how = 'inner')

#set_index = output variable
dtPrincipal = cluster_output.join(dtPrincipal.set_index(campo_output), on = campo_output, how = 'inner')  

#criando novo index
dtPrincipal['new_index'] = list(range(len(dtPrincipal.index)))
dtPrincipal['new_index'] = dtPrincipal['new_index'] + 1
dtPrincipal.set_index(['new_index'], inplace = True)

len(dtPrincipal)

92405

CRIANDO NUMPY ARRAYS COM OS INPUTS E OUTPUTS

In [9]:
#criar um embeddings final que segue a ordem do input no dtPrincipal
embeddingDupl = []
cluster = []

for i in range(1, len(dtPrincipal)):

  linhaEmbedding = (dtPrincipal['input_index'][i]) - 1
  embeddingDupl.append(embeddings_input[linhaEmbedding])
  cluster.append(dtPrincipal['cluster_output'][i]) 
  #cluster.append(dtPrincipal['classes'][i]) #mudar para classes caso queira usar a classe de palavras

len(embeddingDupl)

92404

SEPARANDO INPUT (X) E OUTPUT (Y) COMO NUMPY ARRAY

In [10]:
import numpy as np

X = np.array(embeddingDupl)
Y = np.array(cluster)
X.shape

(92404, 400)

IMPORTS

In [11]:
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


SPLIT DA BASE EM TREINO E TESTE

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

NORMALIZAÇÃO E SCALER

In [13]:
'''
scaler = StandardScaler()
scaler_model = scaler.fit(x_train)

x_train_scaled = scaler_model.transform(x_train)
x_test_scaled = scaler_model.transform(x_test)

pcaComp = PCA(n_components = 0.99)
pcaModel = pcaComp.fit(x_train_scaled)
x_train_prepared = pcaModel.transform(x_train_scaled)
x_test_prepared = pcaModel.transform(x_test_scaled)

'''

#testando sem normalização e Scaler
x_train_prepared = x_train
x_test_prepared = x_test


OBTENDO AS DIMENSÕES

In [14]:
dimensao_x = x_train_prepared.shape[1]
camadas_saida  = len(dtPrincipal['cluster_output'].unique())

print("dimensao x: %(dx)s e dimensao y: %(dy)s"% {'dx': dimensao_x, 'dy': camadas_saida} )

dimensao x: 400 e dimensao y: 7


PARAMETROS MODELOS

In [34]:
epocas = 500

#nomes dos modelos
modelo1 = 'MLP 1 Camada'
modelo2 = 'MLP 2 Camadas'
modelo3 = 'KNN 2'
modelo4 = 'LSTM 1 Camada'
modelo5 = 'LSTM 2 Camadas'
modelo6 = 'LSTM 3 Camadas'

DEFININDO CAMADAS DOS MODELOS

In [16]:
for k in range(3, 10):
  potencia = 2 ** k
  if potencia < dimensao_x:
    primeiro_neuronio_lstm = potencia

for k in range(3, 10):
  potencia = 2 ** k
  if potencia < primeiro_neuronio_lstm:
    segundo_neuronio_lstm = potencia

for k in range(3, 10):
  potencia = 2 ** k
  if potencia < segundo_neuronio_lstm:
    terceiro_neuronio_lstm = potencia

print("O primeiro neuronio do modelo de LSTM tera %(n1)s camadas. O segundo neuronio terá %(n2)s camadas. O terceiro neuronio terá %(n3)s camadas." % {'n1': primeiro_neuronio_lstm, 'n2': segundo_neuronio_lstm, 'n3': terceiro_neuronio_lstm})

O primeiro neuronio do modelo de LSTM tera 256 camadas. O segundo neuronio terá 128 camadas. O terceiro neuronio terá 64 camadas.


In [17]:
primeira_hl_mlp = round((dimensao_x + camadas_saida) / 2)
segunda_hl_mlp = round(primeira_hl_mlp /2)

print("O primeiro neuronio do modelo de MLP tera %(n1)s camadas. O segundo neuronio terá %(n2)s camadas" % {'n1': primeira_hl_mlp, 'n2': segunda_hl_mlp})

O primeiro neuronio do modelo de MLP tera 204 camadas. O segundo neuronio terá 102 camadas




MULTILAYER PERCEPTRON 1 CAMADA


In [18]:
clf_mlp1 = MLPClassifier(solver='sgd'
                     , hidden_layer_sizes=(primeira_hl_mlp)
                     , max_iter = (epocas * 2)
                     , random_state=1)

clf_mlp1.out_activation_ = 'softmax'

clf_mlp1.fit(x_train_prepared, y_train)
y_predicted_mlp1 = clf_mlp1.predict(x_test_prepared)

accuracy_modelo1 = metrics.accuracy_score(y_test, y_predicted_mlp1).round(3)
print(accuracy_modelo1)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.279


In [20]:
mlp1_results = pd.DataFrame(list(zip(y_predicted_mlp1, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(mlp1_results.real, mlp1_results.predito)

export_path = workdir_path + '/matrizConfusaoMLP1.csv'
df_confusion.to_csv (export_path, index = True, header=True)

MULTILAYER PERCEPTRON 2 CAMADAS

In [22]:
clf_mlp2 = MLPClassifier(solver='sgd'
                     , hidden_layer_sizes=(primeira_hl_mlp, segunda_hl_mlp)
                     , max_iter = (epocas * 2)
                     , random_state=1)

clf_mlp2.out_activation_ = 'softmax'

clf_mlp2.fit(x_train_prepared, y_train)
y_predicted_mlp2 = clf_mlp2.predict(x_test_prepared)

accuracy_modelo2 = metrics.accuracy_score(y_test, y_predicted_mlp2).round(3)
print(accuracy_modelo2)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.302


NEAREST NEIGHBOR


In [23]:
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(x_train_prepared, y_train)
y_predicted_neigh = neigh.predict(x_test_prepared)

accuracy_modelo3 = metrics.accuracy_score(y_test, y_predicted_neigh).round(3)
print(accuracy_modelo3)

0.307


RESHAPING PARA USO DOS MODELOS DOS KERAS

In [20]:
#Reshaping
X_train = np.reshape(x_train_prepared, (x_train_prepared.shape[0], 1, x_train_prepared.shape[1]))
X_test = np.reshape(x_test_prepared, (x_test_prepared.shape[0], 1, x_test_prepared.shape[1]))

MODELO DE REDE NEURAL LSTM - 1 CAMADA ESCONDIDA

In [ ]:
#Initicializar a RNN
regressor = Sequential()
 
# Adicionar a primeira camada LSTM e Dropout 
regressor.add(LSTM(units = primeiro_neuronio_lstm, input_shape=(1, x_train_prepared.shape[1])))
regressor.add(Dropout(0.2))
 
# camada de saída
regressor.add(Dense(units = camadas_saida, activation='softmax')) #para classificação dar como entrada as classes com função de ativação softmax

# Compilar a rede
regressor.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Visualizar a rede
regressor.summary()

In [ ]:
regressor.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = epocas, batch_size = 32)

In [27]:
acur_test = regressor.evaluate(X_test, y_test, verbose=0)
accuracy_modelo4 = acur_test[1]
print(accuracy_modelo4)

0.31583788990974426


MATRIZ DE CONFUSÃO

In [28]:
y_predicted_lstm = regressor.predict_classes(X_test)
lstm_results = pd.DataFrame(list(zip(y_predicted_lstm, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(lstm_results.real, lstm_results.predito)

export_path = workdir_path + '/matrizConfusaoLSTM1.csv'
df_confusion.to_csv (export_path, index = True, header=True)

MODELO DE REDES NEURAL LSTM - 2 CAMADAS ESCONDIDAS

In [ ]:
# Initicializar a RNN
regressor = Sequential()
 
# Adicionar a primeira camada LSTM e Dropout 
regressor.add(LSTM(units = primeiro_neuronio_lstm, return_sequences = True, input_shape=(1, x_train_prepared.shape[1])))
regressor.add(Dropout(0.2))
 
# Adicionar a terceira camada LSTM e Dropout
regressor.add(LSTM(units = segundo_neuronio_lstm))
regressor.add(Dropout(0.2))
 
# camada de saída
regressor.add(Dense(units = camadas_saida, activation='softmax')) #para classificação dar como entrada as classes com função de ativação softmax

# Compilar a rede
regressor.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Visualizar a rede
regressor.summary()

In [22]:
#fitting
regressor.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = epocas, batch_size = 32 )

In [23]:
acur_test = regressor.evaluate(X_test, y_test, verbose=0)
accuracy_modelo5 = acur_test[1]
print(accuracy_modelo5)

0.3192467987537384


MATRIZ DE CONFUSÃO

In [24]:
y_predicted_lstm = regressor.predict_classes(X_test)
lstm_results = pd.DataFrame(list(zip(y_predicted_lstm, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(lstm_results.real, lstm_results.predito)

export_path = workdir_path + '/matrizConfusaoLSTM2.csv'
df_confusion.to_csv (export_path, index = True, header=True)

CRIANDO A REDE COM 3 CAMADAS

In [ ]:
# Initicializar a RNN
regressor = Sequential()
 
# Adicionar a primeira camada LSTM e Dropout 
regressor.add(LSTM(units = primeiro_neuronio_lstm, return_sequences = True, input_shape=(1, x_train_prepared.shape[1])))
regressor.add(Dropout(0.2))
 
# Adicionar a segunda camada LSTM e Dropout
regressor.add(LSTM(units = segundo_neuronio_lstm, return_sequences = True))
regressor.add(Dropout(0.2))
 
# Adicionar a terceira camada LSTM e Dropout
regressor.add(LSTM(units = terceiro_neuronio_lstm))
regressor.add(Dropout(0.2))
 
# camada de saída
regressor.add(Dense(units = camadas_saida, activation='softmax')) #para classificação dar como entrada as classes com função de ativação softmax

# Compilar a rede
regressor.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Visualizar a rede
regressor.summary()

In [26]:
#fitting
regressor.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = epocas, batch_size = 32 )

73923/73923 [==============================] - 51s 690us/step - loss: 1.1978 - accuracy: 0.4925 - val_loss: 1.9933 - val_accuracy: 0.3197
Epoch 380/500
73923/73923 [==============================] - 50s 674us/step - loss: 1.1949 - accuracy: 0.4933 - val_loss: 1.9989 - val_accuracy: 0.3164
Epoch 381/500
73923/73923 [==============================] - 52s 704us/step - loss: 1.1926 - accuracy: 0.4903 - val_loss: 2.0007 - val_accuracy: 0.3203
Epoch 382/500
73923/73923 [==============================] - 51s 688us/step - loss: 1.1933 - accuracy: 0.4910 - val_loss: 2.0018 - val_accuracy: 0.3187
Epoch 383/500
73923/73923 [==============================] - 51s 696us/step - loss: 1.1970 - accuracy: 0.4922 - val_loss: 1.9997 - val_accuracy: 0.3175
Epoch 384/500
73923/73923 [==============================] - 51s 691us/step - loss: 1.1956 - accuracy: 0.4914 - val_loss: 2.0007 - val_accuracy: 0.3174
Epoch 385/500
73923/73923 [==============================] - 50s 672us/step - loss: 1.1961 - accuracy:

In [27]:
acur_test = regressor.evaluate(X_test, y_test, verbose=0)
accuracy_modelo6 = acur_test[1]
print(accuracy_modelo6)

0.3165954351425171


MATRIZ DE CONFUSÃO

In [30]:
y_predicted_lstm = regressor.predict_classes(X_test)
lstm_results = pd.DataFrame(list(zip(y_predicted_lstm, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(lstm_results.real, lstm_results.predito)

export_path = workdir_path + '/matrizConfusaoLSTM3.csv'
df_confusion.to_csv (export_path, index = True, header=True)

In [35]:
descricoes_modelos = [modelo1, modelo2, modelo3, modelo4, modelo5, modelo6]
acuracias = [ 0.279, 0.302,  0.307,  0.31583788990974426, accuracy_modelo5, accuracy_modelo6]


EXPORTANDO RESULTADOS DE ACURACIA

In [29]:
descricoes_modelos = [modelo1, modelo2, modelo3, modelo4, modelo5, modelo6]
acuracias = [accuracy_modelo1, accuracy_modelo2, accuracy_modelo3, accuracy_modelo4, accuracy_modelo5, accuracy_modelo6]


NameError: ignored

In [36]:
acuracias_df = pd.DataFrame(list(zip(descricoes_modelos, acuracias)), 
               columns =['modelo', 'acuracia']) 

export_path = workdir_path + '/acuracias.csv'
acuracias_df.to_csv (export_path, index = True, header=True)